# RPLib Problem 0001 - Baseline

Provides the baseline version to rankability problem 0001. Focuses on Massey and Colley out of the box without ties or indirect game information.

In [36]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
import copy
import os

import pandas as pd
import numpy as np

from scipy.stats import pearsonr

from tqdm import tqdm
#import matplotlib.pyplot as plt
from joblib import Parallel, delayed
import joblib
import itertools
from pathlib import Path

from IPython.display import display, Markdown, Latex

**All packages are relative to the home directory of the user**

In [38]:
home = str(Path.home())

**Import the main rankability package**

In [39]:
import sys
sys.path.insert(0,"%s/rankability_toolbox_dev"%home)
import pyrankability

**Load the problem information**

In [40]:
problem = joblib.load("/disk/RPLib/problem_0001.joblib.z")

## Explore and setup the problem

In [41]:
problem.keys()

dict_keys(['description', 'target', 'data', 'other'])

In [42]:
print(problem["description"])


A practitioner wants to predict the degree to which a the rankings during season 
of the NCAA Men’s Basketball are likely to change as more games are played (i.e., sensitivity to more games). 
They want to start the analysis after a minimum of 50% of the games are played. 
They want to run Massey and Colley.

Sensitivity of new games will be measured as the intersection of between two 
rankings derived from before and after the new games are included.



In [43]:
problem['target']

,frac1,frac2,domain,range,direct_thres,spread_thres,weight_indirect,Method,Year,top15_intersection
0,0.5,0.6,all,madness,3.0,3.0,1.0,Colley,2002,0.800000
1,0.5,0.7,all,madness,3.0,3.0,1.0,Colley,2002,0.600000
2,0.5,0.8,all,madness,3.0,3.0,1.0,Colley,2002,0.533333
3,0.5,0.9,all,madness,3.0,3.0,1.0,Colley,2002,0.600000
4,0.5,1.0,all,madness,3.0,3.0,1.0,Colley,2002,0.600000
...,...,...,...,...,...,...,...,...,...,...
3055,0.7,0.9,all,madness,0.0,0.0,0.0,Massey,2018,0.933333
3056,0.7,1.0,all,madness,0.0,0.0,0.0,Massey,2018,0.866667
3057,0.8,0.9,all,madness,0.0,0.0,0.0,Massey,2018,0.866667
3058,0.8,1.0,all,madness,0.0,0.0,0.0,Massey,2018,0.866667


In [44]:
problem['data'].keys()

dict_keys(['2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018'])

In [45]:
problem['data']['2002'].keys()

dict_keys(['frac=0.5', 'frac=0.6', 'frac=0.7', 'frac=0.8', 'frac=0.9', 'frac=1.0'])

**Create easier to reference variables**

In [46]:
years = list(problem['data'].keys())
frac_keys = list(problem['data'][years[0]].keys())
remaining_games = problem['other']['remaining_games']
madness_teams = problem['other']['madness_teams']
best_df = problem['other']['best_df']
top_k = problem['other']['top_k']
target_column = f"top{top_k}_intersection"
best_pred_df = problem['other']['best_pred_df']

## Define helper functions

**Function to compute a D matrix from games using hyperparameters**

In [47]:
def compute_D(game_df,team_range,direct_thres,spread_thres):
    map_func = lambda linked: pyrankability.construct.support_map_vectorized_direct_indirect(linked,direct_thres=direct_thres,spread_thres=spread_thres)
    Ds = pyrankability.construct.V_count_vectorized(game_df,map_func)
    for i in range(len(Ds)):
        Ds[i] = Ds[i].reindex(index=team_range,columns=team_range)
    return Ds

In [48]:
def process(data,target,best_df_all):
    index_cols = ["Year","frac_key","direct_thres","spread_thres","weight_indirect","range"]
    Ds = pd.DataFrame(columns=["D"]+index_cols)
    Ds.set_index(index_cols,inplace=True)
    for frac_key,year in tqdm(itertools.product(frac_keys,years)):
        frac = float(frac_key.split("=")[1])
        best_df = best_df_all.set_index('frac').loc[frac]
        for index,row in best_df.iterrows():
            dom,ran,dt,st,iw = row.loc['domain'],row.loc['range'],row.loc['direct_thres'],row.loc['spread_thres'],row.loc['weight_indirect']
            # set the team_range
            team_range = None
            if ran == 'madness':
                team_range = madness_teams[year]
            elif ran == 'all':
                team_range = all_teams[year]
            else:
                raise Exception(f"range={ran} not supported")
            name = (year,frac_key,dt,st,iw,ran)
            if iw == 0:
                st = np.Inf
            D = compute_D(data[year][frac_key],team_range,dt,st)
            Ds = Ds.append(pd.Series([D],index=["D"],name=name)) 
    return Ds

## Create D matrices
We will ignore indirect games and ties (direct threshold modification).

In [50]:
param_df = best_df.copy()
param_df.spread_thres = 0
param_df.weight_indirect = 0
param_df.direct_thres = 0
param_df

,frac,Method,domain,range,direct_thres,spread_thres,weight_indirect
0,0.5,Colley,all,madness,0,0,0
1,0.5,Massey,all,madness,0,0,0
2,0.6,Colley,all,madness,0,0,0
3,0.6,Massey,all,madness,0,0,0
4,0.7,Colley,all,madness,0,0,0
5,0.7,Massey,all,madness,0,0,0
6,0.8,Colley,all,madness,0,0,0
7,0.8,Massey,all,madness,0,0,0
8,0.9,Colley,all,madness,0,0,0
9,0.9,Massey,all,madness,0,0,0


In [51]:
Ds = process(problem['data'],problem['target'],param_df)

102it [01:33,  1.62s/it]


In [52]:
Ds

D
Year frac_key direct_thres spread_thres weight_indirect range                                                     
2002 frac=0.5 0            0            0               madness  [[Alabama, Alcorn_St, Arizona, Boston_College,...
                                                        madness  [[Alabama, Alcorn_St, Arizona, Boston_College,...
2003 frac=0.5 0            0            0               madness  [[Alabama, Arizona, Arizona_St, Auburn, Austin...
                                                        madness  [[Alabama, Arizona, Arizona_St, Auburn, Austin...
2004 frac=0.5 0            0            0               madness  [[Air_Force, Alabama, Alabama_St, Arizona, BYU...
...                                                                                                            ...
2016 frac=1.0 0            0            0               madness  [[Akron, Arizona, Ark_Little_Rock, Baylor, Bel...
2017 frac=1.0 0            0            0               madness  [[Arizona, Arkansas, Baylor, Bucknell, Butler,...
                                                        madness  [[Arizona, Arkansas, Baylor, Bucknell, Butler,...
2018 frac=1.0 0            0            0               madness  [[Alabama, Arizona, Arkansas, Auburn, Bucknell...
                                                        madness  [[Alabama, Arizona, Arkansas, Auburn, Bucknell...

[204 rows x 1 columns]

In [53]:
Ds.iloc[[0,-1]]

,,,,,,D
Year,frac_key,direct_thres,spread_thres,weight_indirect,range,
2002,frac=0.5,0,0,0,madness,"[[Alabama, Alcorn_St, Arizona, Boston_College,..."
2018,frac=1.0,0,0,0,madness,"[[Alabama, Arizona, Arkansas, Auburn, Bucknell..."


In [54]:
Ds.loc['2002',"D"][0][0]

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  """Entry point for launching an IPython kernel.


team2,Alabama,Alcorn_St,Arizona,Boston_College,Boston_Univ,California,Central_Conn,Charlotte,Cincinnati,Connecticut,...,UNC_Wilmington,USC,Utah,Valparaiso,W_Kentucky,Wake_Forest,Winthrop,Wisconsin,Wyoming,Xavier
team1,,,,,,,,,,,,,,,,,,,,,
Alabama,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Alcorn_St,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Arizona,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
Boston_College,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Boston_Univ,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wake_Forest,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Winthrop,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
Wisconsin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
Ds.loc['2002',"D"][0][1]

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  """Entry point for launching an IPython kernel.


IndexError: list index out of range

In [ ]:
Ds.index.names

### Compute the features

In [40]:
feature_columns = ["delta_lop","delta_hillside","nfrac_xstar_lop","nfrac_xstar_hillside","diameter_lop","diameter_hillside"]

def compute_features(D,rankings,top_k):
    top_teams = list(rankings.sort_values().index[:top_k])
    D = D.loc[top_teams,top_teams]
    
    delta_lop,details_lop = pyrankability.rank.solve(D.fillna(0),method='lop',cont=True)

    x = pd.DataFrame(details_lop['x'],index=D.index,columns=D.columns)
    r = x.sum(axis=0)
    order = np.argsort(r)
    xstar = x.iloc[order,:].iloc[:,order]
    xstar.loc[:,:] = pyrankability.common.threshold_x(xstar.values)
    inxs = np.triu_indices(len(xstar),k=1)
    xstar_upper = xstar.values[inxs[0],inxs[1]]
    nfrac_upper_lop = sum((xstar_upper > 0) & (xstar_upper < 1))
    
    top_teams = xstar.columns[:top_k]
    
    k_two_distant,details_two_distant = pyrankability.search.solve_pair_max_tau(D.fillna(0),method='lop',cont=False,verbose=False)
    d_lop = details_two_distant['tau']
    
    delta_hillside,details_hillside = pyrankability.rank.solve(D,method='hillside',cont=True)
    
    x = pd.DataFrame(details_hillside['x'],index=D.index,columns=D.columns)
    r = x.sum(axis=0)
    order = np.argsort(r)
    xstar = x.iloc[order,:].iloc[:,order]
    xstar.loc[:,:] = pyrankability.common.threshold_x(xstar.values)
    inxs = np.triu_indices(len(xstar),k=1)
    xstar_upper = xstar.values[inxs[0],inxs[1]]
    nfrac_upper_hillside = sum((xstar_upper > 0) & (xstar_upper < 1))
    
    top_teams = xstar.columns[:top_k]
    
    k_two_distant,details_two_distant = pyrankability.search.solve_pair_max_tau(D,method='hillside',verbose=False,cont=False)
    d_hillside = details_two_distant['tau']
    
    features = pd.Series([delta_lop,delta_hillside,2*nfrac_upper_lop,2*nfrac_upper_hillside,d_lop,d_hillside],index=feature_columns)

    return features

In [41]:
best_pred_df = best_pred_df.reset_index()
best_pred_df['frac_key'] = "frac="+best_pred_df['frac'].astype(str)
best_pred_df

,domain,range,direct_thres,spread_thres,weight_indirect,frac,Method,Year,Predictability,rankings,frac_key
0,all,madness,3.0,3.0,1.0,0.5,Colley,2002,64.044944,Alabama 31.0 Alcorn_St 64.0 ...,frac=0.5
1,all,madness,3.0,3.0,1.0,0.5,Colley,2003,65.476190,Alabama 8.0 Arizona 2.0 Arizon...,frac=0.5
2,all,madness,3.0,3.0,1.0,0.5,Colley,2004,58.064516,Air_Force 39.0 Alabama 23.0 Alabam...,frac=0.5
3,all,madness,3.0,3.0,1.0,0.5,Colley,2005,61.538462,Alabama 23.0 Alabama_A&M 65.0 ...,frac=0.5
4,all,madness,3.0,3.0,1.0,0.5,Colley,2006,68.604651,Air_Force 26.0 Alabama 48.0 Al...,frac=0.5
...,...,...,...,...,...,...,...,...,...,...,...
199,all,madness,0.0,0.0,0.0,1.0,Massey,2014,65.000000,Albany_NY 64.0 American_Univ 59.0 Ar...,frac=1.0
200,all,madness,0.0,0.0,0.0,1.0,Massey,2015,69.387755,Albany_NY 61.0 Arizona 3.0 Arkansas...,frac=1.0
201,all,madness,0.0,0.0,0.0,1.0,Massey,2016,70.238095,Akron 53.0 Arizona 12....,frac=1.0
202,all,madness,0.0,0.0,0.0,1.0,Massey,2017,54.838710,Arizona 26.0 Arkansas 35.0 Ba...,frac=1.0


In [42]:
def create_features(Ds,best_pred_df,top_k):
    index_cols = list(Ds.index.names)+["Method","Construction"]
    X = pd.DataFrame(columns=index_cols + feature_columns)
    X.set_index(index_cols,inplace=True)
    for index,row in tqdm(Ds.iterrows()):
        year,frac_key,dt,st,iw,ran = index
        frac = float(frac_key.split("=")[1])
        spec_best_pred_df = best_pred_df.set_index(list(Ds.index.names)).loc[[(year,frac_key,dt,st,iw,ran)]]
        sum_D = None
        for i,D in enumerate(Ds.loc[(year,frac_key,dt,st,iw,ran),"D"]):
            if sum_D is None:
                sum_D = D
            else:
                sum_D = sum_D.add(iw*D,fill_value=0)
            if i == 0:
                construction = "Direct"
            elif i == 1:
                construction = "Indirect"
            else:
                raise Exception("Error")
            methods = spec_best_pred_df["Method"].unique()
            for method in methods:
                rankings = spec_best_pred_df.set_index('Method').loc[method,'rankings']
                features = compute_features(D,rankings,top_k)
                features.name = tuple(list(index)+[method,construction])
                X = X.append(features)
        construction = "Both"
        methods = spec_best_pred_df["Method"].unique()
        for method in methods:
            rankings = spec_best_pred_df.set_index('Method').loc[method,'rankings']
            features = compute_features(sum_D,rankings,top_k)
            features.name = tuple(list(index)+[method,construction])
            X = X.append(features)
    return X

In [87]:
X = create_features(Ds,best_pred_df.reset_index(),top_k)














0it [00:00, ?it/s]












1it [00:00,  1.06it/s]












2it [00:01,  1.26it/s]












3it [00:02,  1.23it/s]












4it [00:02,  1.34it/s]












5it [00:03,  1.35it/s]












6it [00:04,  1.53it/s]












7it [00:04,  1.40it/s]












8it [00:05,  1.57it/s]












9it [00:06,  1.47it/s]












10it [00:06,  1.63it/s]












11it [00:07,  1.53it/s]












12it [00:07,  1.63it/s]












13it [00:08,  1.56it/s]












14it [00:09,  1.54it/s]












15it [00:10,  1.44it/s]












16it [00:10,  1.64it/s]












17it [00:11,  1.52it/s]












18it [00:11,  1.68it/s]












19it [00:12,  1.57it/s]












20it [00:12,  1.74it/s]












21it [00:13,  1.52it/s]












22it [00:14,  1.36it/s]












23it [00:15,  1.35it/s]












24it [00:15,  1.52it/s]












25it [00:16,  1.38it/s]












26it [00:17,  1.56it/s]












27it [00:17,  1.52it/s]







In [88]:
X

delta_lop  \
Year frac_key direct_thres spread_thres weight_indirect range   Method Construction              
2002 frac=0.5 3.0          3.0          1.0             madness Colley Direct              0.0   
                                                                       Indirect            4.0   
                                                                       Both                8.0   
              0.0          0.0          0.0             madness Massey Direct              0.0   
                                                                       Both                0.0   
...                                                                                        ...   
2018 frac=1.0 0.0          3.0          1.0             madness Colley Direct              6.0   
                                                                       Indirect           42.0   
                                                                       Both               57.0   
                           0.0          0.0             madness Massey Direct              7.0   
                                                                       Both                7.0   

                                                                                     delta_hillside  \
Year frac_key direct_thres spread_thres weight_indirect range   Method Construction                   
2002 frac=0.5 3.0          3.0          1.0             madness Colley Direct                   0.0   
                                                                       Indirect                90.0   
                                                                       Both                   127.0   
              0.0          0.0          0.0             madness Massey Direct                   0.0   
                                                                       Both                     0.0   
...                                                                                             ...   
2018 frac=1.0 0.0          3.0          1.0             madness Colley Direct                   8.0   
                                                                       Indirect               473.0   
                                                                       Both                   522.0   
                           0.0          0.0             madness Massey Direct                   7.0   
                                                                       Both                     7.0   

                                                                                     nfrac_xstar_lop  \
Year frac_key direct_thres spread_thres weight_indirect range   Method Construction                    
2002 frac=0.5 3.0          3.0          1.0             madness Colley Direct                  148.0   
                                                                       Indirect                124.0   
                                                                       Both                    162.0   
              0.0          0.0          0.0             madness Massey Direct                  192.0   
                                                                       Both                    192.0   
...                                                                                              ...   
2018 frac=1.0 0.0          3.0          1.0             madness Colley Direct                  166.0   
                                                                       Indirect                 16.0   
                                                                       Both                     74.0   
                           0.0          0.0             madness Massey Direct                  150.0   
                                                                       Both                    150.0   

                                                                                     nfrac_xstar_hillside  \
Year fra

## Refine the target dataset

In [89]:
target = problem['target'].groupby(['frac1','frac2','Method','Year','direct_thres','spread_thres','weight_indirect'])[target_column].mean().to_frame()
target

top15_intersection
frac1 frac2 Method Year direct_thres spread_thres weight_indirect                    
0.5   0.6   Colley 2002 0.0          0.0          0.5                        0.800000
                                     3.0          0.5                        0.800000
                                                  1.0                        0.800000
                        3.0          0.0          1.0                        0.800000
                                     3.0          1.0                        0.800000
...                                                                               ...
0.9   1.0   Massey 2016 3.0          0.0          0.0                        0.933333
                   2017 0.0          0.0          0.0                        0.866667
                        3.0          0.0          0.0                        0.866667
                   2018 0.0          0.0          0.0                        0.866667
                        3.0          0.0          0.0                        0.866667

[1785 rows x 1 columns]

In [90]:
X_for_join = X.copy().reset_index()
X_for_join['frac1']= X_for_join['frac_key'].str.replace("frac=","").astype(float)
X_for_join

,Year,frac_key,direct_thres,spread_thres,weight_indirect,range,Method,Construction,delta_lop,delta_hillside,nfrac_xstar_lop,nfrac_xstar_hillside,diameter_lop,diameter_hillside,frac1
0,2002,frac=0.5,3.0,3.0,1.0,madness,Colley,Direct,0.0,0.0,148.0,178.0,-0.409524,-0.695238,0.5
1,2002,frac=0.5,3.0,3.0,1.0,madness,Colley,Indirect,4.0,90.0,124.0,32.0,-0.180952,0.714286,0.5
2,2002,frac=0.5,3.0,3.0,1.0,madness,Colley,Both,8.0,127.0,162.0,14.0,-0.428571,0.866667,0.5
3,2002,frac=0.5,0.0,0.0,0.0,madness,Massey,Direct,0.0,0.0,192.0,206.0,-0.828571,-0.961905,0.5
4,2002,frac=0.5,0.0,0.0,0.0,madness,Massey,Both,0.0,0.0,192.0,206.0,-0.828571,-0.961905,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505,2018,frac=1.0,0.0,3.0,1.0,madness,Colley,Direct,6.0,8.0,166.0,156.0,-0.561905,-0.447619,1.0
506,2018,frac=1.0,0.0,3.0,1.0,madness,Colley,Indirect,42.0,473.0,16.0,8.0,0.847619,0.923810,1.0
507,2018,frac=1.0,0.0,3.0,1.0,madness,Colley,Both,57.0,522.0,74.0,10.0,0.333333,0.904762,1.0
508,2018,frac=1.0,0.0,0.0,0.0,madness,Massey,Direct,7.0,7.0,150.0,124.0,-0.409524,-0.123810,1.0


In [91]:
target

top15_intersection
frac1 frac2 Method Year direct_thres spread_thres weight_indirect                    
0.5   0.6   Colley 2002 0.0          0.0          0.5                        0.800000
                                     3.0          0.5                        0.800000
                                                  1.0                        0.800000
                        3.0          0.0          1.0                        0.800000
                                     3.0          1.0                        0.800000
...                                                                               ...
0.9   1.0   Massey 2016 3.0          0.0          0.0                        0.933333
                   2017 0.0          0.0          0.0                        0.866667
                        3.0          0.0          0.0                        0.866667
                   2018 0.0          0.0          0.0                        0.866667
                        3.0          0.0          0.0                        0.866667

[1785 rows x 1 columns]

In [92]:
Xy = target.reset_index().set_index(['Method','frac1','Year','direct_thres','spread_thres','weight_indirect']).join(X_for_join.set_index(['Method','frac1','Year','direct_thres','spread_thres','weight_indirect'])).dropna()
Xy = Xy.reset_index()
Xy

,Method,frac1,Year,direct_thres,spread_thres,weight_indirect,frac2,top15_intersection,frac_key,range,Construction,delta_lop,delta_hillside,nfrac_xstar_lop,nfrac_xstar_hillside,diameter_lop,diameter_hillside
0,Colley,0.5,2002,3.0,3.0,1.0,0.6,0.800000,frac=0.5,madness,Direct,0.0,0.0,148.0,178.0,-0.409524,-0.695238
1,Colley,0.5,2002,3.0,3.0,1.0,0.6,0.800000,frac=0.5,madness,Indirect,4.0,90.0,124.0,32.0,-0.180952,0.714286
2,Colley,0.5,2002,3.0,3.0,1.0,0.6,0.800000,frac=0.5,madness,Both,8.0,127.0,162.0,14.0,-0.428571,0.866667
3,Colley,0.5,2002,3.0,3.0,1.0,0.7,0.600000,frac=0.5,madness,Direct,0.0,0.0,148.0,178.0,-0.409524,-0.695238
4,Colley,0.5,2002,3.0,3.0,1.0,0.7,0.600000,frac=0.5,madness,Indirect,4.0,90.0,124.0,32.0,-0.180952,0.714286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1270,Massey,0.9,2016,3.0,0.0,0.0,1.0,0.933333,frac=0.9,madness,Both,4.0,12.0,128.0,96.0,-0.219048,0.142857
1271,Massey,0.9,2017,3.0,0.0,0.0,1.0,0.866667,frac=0.9,madness,Direct,3.0,20.0,160.0,72.0,-0.504762,0.333333
1272,Massey,0.9,2017,3.0,0.0,0.0,1.0,0.866667,frac=0.9,madness,Both,3.0,20.0,160.0,72.0,-0.504762,0.333333
1273,Massey,0.9,2018,3.0,0.0,0.0,1.0,0.866667,frac=0.9,madness,Direct,2.0,13.0,134.0,76.0,-0.276190,0.276190


## Process results

In [93]:
frac_pairs = [(0.5,0.6),(0.6,0.7),(0.7,0.8),(0.8,0.9),(0.9,1.)]
summary = None
for pair in frac_pairs:
    data = Xy.set_index(['frac1','frac2']).loc[pair].reset_index()
    for_corr = data.set_index(['Method','Construction',"frac1","frac2"])
    if summary is None:
        summary = pd.DataFrame(columns=["frac1","frac2","Method","Construction"]+feature_columns).set_index(list(for_corr.index.names))
    for ix in for_corr.index.unique():
        corr_results = for_corr.loc[ix][[target_column]+feature_columns].corr()
        target_corr_results = corr_results.loc[target_column].drop(target_column)
        target_corr_results.name = ix
        summary = summary.append(target_corr_results)

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:4: PerformanceWarning: indexing past lexsort depth may impact performance.
  after removing the cwd from sys.path.
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:9: PerformanceWarning: indexing past lexsort depth may impact performance.
  if __name__ == '__main__':


In [94]:
display(summary)

delta_lop  delta_hillside  nfrac_xstar_lop  \
Method Construction frac1 frac2                                               
Colley Direct       0.5   0.6    -0.112962       -0.127082        -0.108058   
       Indirect     0.5   0.6    -0.158208       -0.062079        -0.305590   
       Both         0.5   0.6    -0.295519       -0.097597        -0.461342   
Massey Direct       0.5   0.6     0.094033        0.083245        -0.053071   
       Both         0.5   0.6     0.094033        0.083245        -0.053071   
Colley Direct       0.6   0.7    -0.063766       -0.233558        -0.652818   
       Indirect     0.6   0.7     0.234369        0.199114        -0.106610   
       Both         0.6   0.7     0.164411        0.269190        -0.199390   
Massey Direct       0.6   0.7     0.434428        0.355113         0.131484   
       Both         0.6   0.7     0.434428        0.355113         0.131484   
Colley Direct       0.7   0.8     0.458542        0.443589        -0.176383   
       Indirect     0.7   0.8     0.291448        0.266450         0.020887   
       Both         0.7   0.8     0.293376        0.247042        -0.636100   
Massey Direct       0.7   0.8     0.413636        0.078820         0.411390   
       Both         0.7   0.8     0.413636        0.078820         0.411390   
Colley Direct       0.8   0.9     0.095538       -0.286339         0.428276   
       Indirect     0.8   0.9     0.020985       -0.049858         0.272791   
       Both         0.8   0.9     0.009634       -0.076165         0.231198   
Massey Direct       0.8   0.9    -0.319991       -0.117305         0.479787   
       Both         0.8   0.9    -0.319991       -0.117305         0.479787   
Colley Direct       0.9   1.0     0.155742        0.213578        -0.232564   
       Indirect     0.9   1.0     0.316502        0.534517        -0.253307   
       Both         0.9   1.0     0.285571        0.566860        -0.085335   
Massey Direct       0.9   1.0     0.012557       -0.063081        -0.129400   
       Both         0.9   1.0     0.012557       -0.063081        -0.129400   

                                 nfrac_xstar_hillside  diameter_lop  \
Method Construction frac1 frac2                                       
Colley Direct       0.5   0.6               -0.101531      0.093924   
       Indirect     0.5   0.6               -0.226777      0.282375   
       Both         0.5   0.6               -0.381028      0.472672   
Massey Direct       0.5   0.6               -0.166145      0.089210   
       Both         0.5   0.6               -0.166145      0.089210   
Colley Direct       0.6   0.7               -0.561850      0.657673   
       Indirect     0.6   0.7               -0.149980      0.166488   
       Both         0.6   0.7               -0.405003      0.244061   
Massey Direct       0.6   0.7                0.116638     -0.091029   
       Both         0.6   0.7                0.116638     -0.091029   
Colley Direct       0.7   0.8               -0.190110      0.177562   
       Indirect     0.7   0.8               -0.177398      0.032778   
       Both         0.7   0.8                0.068415      0.654260   
Massey Direct       0.7   0.8                0.207175     -0.369675   
       Both         0.7   0.8                0.207175     -0.369675   
Colley Direct       0.8   0.9                0.332149     -0.429195   
       Indirect     0.8   0.9                0.075405     -0.256095   
       Both         0.8   0.9               -0.364784     -0.232181   
Massey Direct       0.8   0.9                0.433197     -0.484527   
       Both         0.8   0.9                0.433197     -0.484527   
Colley Direct       0.9   1.0               -0.461277      0.266511   
       Indirect     0.9   1.0                0.065777      0.299302   
       Both         0.9   1.0                0.092812      0.072017   
Massey Direct       0.9   1.0               -0.035208      0.146855   
       Both         0.9   1.0               -0.035208

In [82]:
summary

,,,,delta_lop,delta_hillside,nfrac_xstar_lop,nfrac_xstar_hillside,diameter_lop,diameter_hillside
Method,Construction,frac1,frac2,,,,,,
Colley,Direct,0.5,0.6,1.809442e-16,0.218595,-0.274607,-0.34159,0.304669,0.313927


## 0.6 to 0.7

In [57]:
data = Xy.set_index(['frac1','frac2']).loc[(0.6,0.7)].reset_index()
for_corr = data.set_index(['Method','Construction'])
for ix in for_corr.index.unique():
    display(pd.Series(ix,index=for_corr.index.names))
    display(for_corr.loc[ix][[target_column]+feature_columns].corr())

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  """Entry point for launching an IPython kernel.


Method          Colley
Construction    Direct
dtype: object

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  """


,top15_intersection,delta_lop,delta_hillside,nfrac_xstar_lop,nfrac_xstar_hillside,diameter_lop,diameter_hillside
top15_intersection,1.000000,-0.033692,0.034758,-0.124823,0.229806,0.275894,-0.168381
delta_lop,-0.033692,1.000000,0.735085,0.279614,-0.197032,-0.099624,0.292868
delta_hillside,0.034758,0.735085,1.000000,0.341162,-0.555433,-0.315206,0.610122
nfrac_xstar_lop,-0.124823,0.279614,0.341162,1.000000,0.070455,-0.931563,-0.051844
nfrac_xstar_hillside,0.229806,-0.197032,-0.555433,0.070455,1.000000,0.066163,-0.973198
diameter_lop,0.275894,-0.099624,-0.315206,-0.931563,0.066163,1.000000,-0.045087
diameter_hillside,-0.168381,0.292868,0.610122,-0.051844,-0.973198,-0.045087,1.000000


Method            Colley
Construction    Indirect
dtype: object

...

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  """


,top15_intersection,delta_lop,delta_hillside,nfrac_xstar_lop,nfrac_xstar_hillside,diameter_lop,diameter_hillside
top15_intersection,1.000000,0.051682,0.070177,0.323252,0.000470,-0.328800,-0.114328
delta_lop,0.051682,1.000000,0.682160,0.546261,-0.311122,-0.566135,0.339730
delta_hillside,0.070177,0.682160,1.000000,0.059334,-0.608424,-0.139715,0.611381
nfrac_xstar_lop,0.323252,0.546261,0.059334,1.000000,0.420533,-0.978047,-0.422727
nfrac_xstar_hillside,0.000470,-0.311122,-0.608424,0.420533,1.000000,-0.426080,-0.979986
diameter_lop,-0.328800,-0.566135,-0.139715,-0.978047,-0.426080,1.000000,0.431566
diameter_hillside,-0.114328,0.339730,0.611381,-0.422727,-0.979986,0.431566,1.000000


Method          Massey
Construction      Both
dtype: object

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  """


,top15_intersection,delta_lop,delta_hillside,nfrac_xstar_lop,nfrac_xstar_hillside,diameter_lop,diameter_hillside
top15_intersection,1.000000,0.051682,0.070177,0.323252,0.000470,-0.328800,-0.114328
delta_lop,0.051682,1.000000,0.682160,0.546261,-0.311122,-0.566135,0.339730
delta_hillside,0.070177,0.682160,1.000000,0.059334,-0.608424,-0.139715,0.611381
nfrac_xstar_lop,0.323252,0.546261,0.059334,1.000000,0.420533,-0.978047,-0.422727
nfrac_xstar_hillside,0.000470,-0.311122,-0.608424,0.420533,1.000000,-0.426080,-0.979986
diameter_lop,-0.328800,-0.566135,-0.139715,-0.978047,-0.426080,1.000000,0.431566
diameter_hillside,-0.114328,0.339730,0.611381,-0.422727,-0.979986,0.431566,1.000000


### 0.7 to 0.8

In [58]:
data = Xy.set_index(['frac1','frac2']).loc[(0.7,0.8)].reset_index()
for_corr = data.set_index(['Method'])
for ix in for_corr.index.unique():
    display(pd.Series(ix,index=for_corr.index.names))
    display(for_corr.loc[ix][[target_column]+feature_columns].corr())

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  """Entry point for launching an IPython kernel.


Method    Colley
dtype: object

,top15_intersection,delta_lop,delta_hillside,nfrac_xstar_lop,nfrac_xstar_hillside,diameter_lop,diameter_hillside
top15_intersection,1.000000,0.022290,0.023643,-0.145952,-0.019798,0.131875,0.013795
delta_lop,0.022290,1.000000,0.856411,-0.600197,-0.833061,0.699814,0.840072
delta_hillside,0.023643,0.856411,1.000000,-0.823404,-0.923045,0.869911,0.929844
nfrac_xstar_lop,-0.145952,-0.600197,-0.823404,1.000000,0.778909,-0.974619,-0.795137
nfrac_xstar_hillside,-0.019798,-0.833061,-0.923045,0.778909,1.000000,-0.854231,-0.996746
diameter_lop,0.131875,0.699814,0.869911,-0.974619,-0.854231,1.000000,0.869986
diameter_hillside,0.013795,0.840072,0.929844,-0.795137,-0.996746,0.869986,1.000000


Method    Massey
dtype: object

,top15_intersection,delta_lop,delta_hillside,nfrac_xstar_lop,nfrac_xstar_hillside,diameter_lop,diameter_hillside
top15_intersection,1.000000,0.244961,0.302184,-0.191937,-0.126197,0.204923,0.362996
delta_lop,0.244961,1.000000,0.569115,0.359219,-0.127186,-0.413252,0.369689
delta_hillside,0.302184,0.569115,1.000000,0.049907,-0.061282,-0.080169,0.456502
nfrac_xstar_lop,-0.191937,0.359219,0.049907,1.000000,0.306418,-0.966973,-0.333667
nfrac_xstar_hillside,-0.126197,-0.127186,-0.061282,0.306418,1.000000,-0.301968,-0.789993
diameter_lop,0.204923,-0.413252,-0.080169,-0.966973,-0.301968,1.000000,0.373802
diameter_hillside,0.362996,0.369689,0.456502,-0.333667,-0.789993,0.373802,1.000000


### 0.8 to 0.9

In [59]:
data = Xy.set_index(['frac1','frac2']).loc[(0.8,0.9)].reset_index()
for_corr = data.set_index(['Method'])
for ix in for_corr.index.unique():
    display(pd.Series(ix,index=for_corr.index.names))
    display(for_corr.loc[ix][[target_column]+feature_columns].corr())

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  """Entry point for launching an IPython kernel.


Method    Colley
dtype: object

,top15_intersection,delta_lop,delta_hillside,nfrac_xstar_lop,nfrac_xstar_hillside,diameter_lop,diameter_hillside
top15_intersection,1.000000,-0.063546,-0.046630,0.021394,0.072194,-0.033096,-0.069062
delta_lop,-0.063546,1.000000,0.865985,-0.624501,-0.781810,0.731686,0.801398
delta_hillside,-0.046630,0.865985,1.000000,-0.833724,-0.913166,0.884851,0.928728
nfrac_xstar_lop,0.021394,-0.624501,-0.833724,1.000000,0.778140,-0.964683,-0.805747
nfrac_xstar_hillside,0.072194,-0.781810,-0.913166,0.778140,1.000000,-0.836519,-0.987047
diameter_lop,-0.033096,0.731686,0.884851,-0.964683,-0.836519,1.000000,0.868255
diameter_hillside,-0.069062,0.801398,0.928728,-0.805747,-0.987047,0.868255,1.000000


Method    Massey
dtype: object

,top15_intersection,delta_lop,delta_hillside,nfrac_xstar_lop,nfrac_xstar_hillside,diameter_lop,diameter_hillside
top15_intersection,1.000000,-0.200329,-0.081585,0.151543,-0.029163,-0.148642,0.088628
delta_lop,-0.200329,1.000000,0.410175,0.611958,-0.035864,-0.526587,-0.072341
delta_hillside,-0.081585,0.410175,1.000000,0.046915,-0.104072,-0.051227,0.012704
nfrac_xstar_lop,0.151543,0.611958,0.046915,1.000000,0.295671,-0.974192,-0.275981
nfrac_xstar_hillside,-0.029163,-0.035864,-0.104072,0.295671,1.000000,-0.361536,-0.901983
diameter_lop,-0.148642,-0.526587,-0.051227,-0.974192,-0.361536,1.000000,0.351146
diameter_hillside,0.088628,-0.072341,0.012704,-0.275981,-0.901983,0.351146,1.000000


### 0.9 to 1.

In [60]:
data = Xy.set_index(['frac1','frac2']).loc[(0.9,1.)].reset_index()
for_corr = data.set_index(['Method'])
for ix in for_corr.index.unique():
    display(pd.Series(ix,index=for_corr.index.names))
    display(for_corr.loc[ix][[target_column]+feature_columns].corr())

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  """Entry point for launching an IPython kernel.


Method    Colley
dtype: object

,top15_intersection,delta_lop,delta_hillside,nfrac_xstar_lop,nfrac_xstar_hillside,diameter_lop,diameter_hillside
top15_intersection,1.000000,0.080682,0.101705,-0.090726,-0.104757,0.084122,0.138211
delta_lop,0.080682,1.000000,0.865931,-0.664981,-0.747569,0.736892,0.760151
delta_hillside,0.101705,0.865931,1.000000,-0.839537,-0.861560,0.889460,0.879020
nfrac_xstar_lop,-0.090726,-0.664981,-0.839537,1.000000,0.751581,-0.973630,-0.806614
nfrac_xstar_hillside,-0.104757,-0.747569,-0.861560,0.751581,1.000000,-0.811210,-0.926384
diameter_lop,0.084122,0.736892,0.889460,-0.973630,-0.811210,1.000000,0.875447
diameter_hillside,0.138211,0.760151,0.879020,-0.806614,-0.926384,0.875447,1.000000


Method    Massey
dtype: object

,top15_intersection,delta_lop,delta_hillside,nfrac_xstar_lop,nfrac_xstar_hillside,diameter_lop,diameter_hillside
top15_intersection,1.000000,0.060080,0.144706,-0.271090,-0.074017,0.379677,0.254621
delta_lop,0.060080,1.000000,0.530237,0.151979,0.074900,0.052209,0.195383
delta_hillside,0.144706,0.530237,1.000000,-0.453008,-0.129820,0.488829,0.367424
nfrac_xstar_lop,-0.271090,0.151979,-0.453008,1.000000,0.181947,-0.909558,-0.203733
nfrac_xstar_hillside,-0.074017,0.074900,-0.129820,0.181947,1.000000,-0.157968,-0.883072
diameter_lop,0.379677,0.052209,0.488829,-0.909558,-0.157968,1.000000,0.297851
diameter_hillside,0.254621,0.195383,0.367424,-0.203733,-0.883072,0.297851,1.000000


In [72]:
for_corr = data.set_index(['Method','direct_thres','spread_thres','weight_indirect'])
for_display = pd.DataFrame(columns=feature_columns+list(for_corr.index.names))
for_display.set_index(list(for_corr.index.names),inplace=True)
for ix in for_corr.index.unique():
    dt = for_corr.loc[ix][[target_column]+feature_columns].corr().loc[target_column,feature_columns]
    dt.name = ix
    for_display = for_display.append(dt)

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  """


In [77]:
for_display.T

Method,Massey,Colley
direct_thres,0.0,3.0
spread_thres,3.0,3.0
weight_indirect,0.25,0.00
delta_lop,-0.181122,-0.045966
delta_hillside,-0.118407,0.284547
nfrac_xstar_lop,0.067466,-0.065937
nfrac_xstar_hillside,-0.087980,0.058396
diameter_lop,-0.079867,0.376589
diameter_hillside,0.421510,0.102595


In [76]:
print(for_display.T.to_latex())

\begin{tabular}{lrr}
\toprule
Method &    Massey &    Colley \\
direct\_thres &       0.0 &       3.0 \\
spread\_thres &       3.0 &       3.0 \\
weight\_indirect &      0.25 &      0.00 \\
\midrule
delta\_lop            & -0.181122 & -0.045966 \\
delta\_hillside       & -0.118407 &  0.284547 \\
nfrac\_xstar\_lop      &  0.067466 & -0.065937 \\
nfrac\_xstar\_hillside & -0.087980 &  0.058396 \\
diameter\_lop         & -0.079867 &  0.376589 \\
diameter\_hillside    &  0.421510 &  0.102595 \\
\bottomrule
\end{tabular}

